In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42) #seed 고정

/Users/yeong-gwang/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')


display(train.head(3))
display(test.head(3)) 


,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860


,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status
0,TEST_0000,79,M,High Junior,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,...,Householder,Native,US,Unknown,Unknown,Single,0,0,0,Under Median
1,TEST_0001,47,M,Elementary (5-6),Children or Armed Forces,0,Not in universe or children,Unknown,White,Other Spanish,...,Child 18 or older,Native,US,US,US,Nonfiler,0,0,0,Under Median
2,TEST_0002,18,F,High Junior,Children or Armed Forces,52,Retail,Services,White,All other,...,Child 18 or older,Native,US,US,US,Single,0,0,0,Under Median


In [3]:
train_x= train.drop(columns=['ID', 'Income'])
train_y = train['Income']

test_x= test.drop(columns=['ID'])


In [4]:
from sklearn.preprocessing import LabelEncoder

encoding_target = list(train_x.dtypes[train_x.dtypes == 'object'].index)

for i in encoding_target:
    le = LabelEncoder()
    
    #train과 test 데이터셋에서 해당 열의 모든 값을 문자열로 변환
    train_x[i]=train_x[i].astype(str)
    test_x[i]=test_x[i].astype(str)
    
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    #test 데이터의 새로운 카테고리에 대해 le.classes_ 배열에 추가 
    for case in np.unique(test_x[i]):
        if case not in le.classes_:
            le.classes_ = np.append(le.classes_, case)
            
    test_x[i] = le.transform(test_x[i])
    
    

In [5]:
from sklearn.model_selection import train_test_split                                                     

X_train, X_valid, y_train, y_valid = train_test_split(train_x, 
                                                      train_y, 
                                                      test_size=0.2, 
                                                      random_state=42)

# XGBRegressor

In [6]:
from xgboost import XGBRegressor
from math import sqrt

import warnings
warnings.filterwarnings(action='ignore')

xgb_reg = XGBRegressor(n_estimators=1000, random_state=0)
xgb_reg.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error
pred = xgb_reg.predict(X_valid)
rmse = sqrt(mean_squared_error(y_valid,np.round(pred)))

print(f'XGBRegressor RMSE: {rmse:.2f}')


XGBRegressor RMSE: 667.35


# AdaBoostRegressor

In [7]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# AdaBoostRegressor 모델 생성 및 학습
ab_reg = AdaBoostRegressor(random_state=0, n_estimators=100)
ab_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = ab_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid,np.round(pred)))

print(f'AdaBoostRegressor RMSE: {rmse:.2f}')

AdaBoostRegressor RMSE: 719.85


# Ridge

In [8]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt

# Ridge 회귀 모델 생성 및 학습
rd_reg = Ridge(random_state=0)
rd_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = rd_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid,np.round(pred)))

print(f'Ridge Regression RMSE: {rmse:.2f}')


Ridge Regression RMSE: 624.05


# BaggingRegressor

In [9]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# BaggingRegressor 모델 생성 및 학습
bg_reg = BaggingRegressor(random_state=0)
bg_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = bg_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid,np.round(pred)))

print(f'BaggingRegressor RMSE: {rmse:.2f}')


BaggingRegressor RMSE: 636.24


# DecisionTreeRegressor

In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# DecisionTreeRegressor 모델 생성 및 학습
dt_reg = DecisionTreeRegressor(random_state=0)
dt_reg.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
pred = dt_reg.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid,np.round(pred)))
print(f'DecisionTreeRegressor RMSE: {rmse:.2f}')


DecisionTreeRegressor RMSE: 840.31


# SUMMARY

In [11]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# 모델 리스트 생성 (이전에 정의된 회귀 모델들)
model_lst = [xgb_reg, ab_reg, rd_reg, bg_reg, dt_reg]

# 각 모델에 대한 RMSE 계산 및 출력
for model in model_lst:
    pred = model.predict(X_valid)  # 검증 데이터셋에 대한 예측 수행
    rmse = sqrt(mean_squared_error(y_valid,np.round(pred)))
    class_name = model.__class__.__name__  # 모델 클래스 이름
    print(f'{class_name} RMSE: {rmse:.2f}')


XGBRegressor RMSE: 667.35
AdaBoostRegressor RMSE: 719.85
Ridge RMSE: 624.05
BaggingRegressor RMSE: 636.24
DecisionTreeRegressor RMSE: 840.31


전처리를 진행한 것과 진행하지 않고 모델만 돌렸을 때 rmse 값이 더 낮게 나타났다.
